<a href="https://colab.research.google.com/github/julesprotacio/ACMDIGW-Files/blob/main/OG_Extracting_and_Saving_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Start of ACMDIGW
#Start of extracting and saving data using Yelp API

#crtl + / = comment
# ctrl + enter = run code
# crtl + m, b = add code block below
# ctrl + m, a = add code block above


In [ ]:
#connect GDRive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# install yelpapi
!pip install yelpapi --quiet

In [ ]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#additional libraries
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

Loading Credentials and Creating a YelpAPI Object



In [ ]:
# load yelp API Credentials and instantiate the  Yelp API object
with open('/content/drive/MyDrive/Colab Notebooks/credentials/yelp_api.json') as file:
  yelp_credentials = json.load(file)

# create the yelp api engine
yelp_api = YelpAPI(yelp_credentials)
yelp_api = YelpAPI (yelp_credentials ['api-key'], timeout_s=5.0)



In [ ]:
#define API cell parameters and output file path
LOCATION = 'Greenville, SC'
TERM = 'Sushi'
JSON_FILE = '/Data/results_SC_Sushi.json'


In [ ]:
#display the destination file path
print(f'Data will be saved to:{JSON_FILE}')

Data will be saved to:/Data/results_SC_Sushi.json


In [ ]:
#check if JSON_FILE exists and create if it doesn't
if not os.path.isfile (JSON_FILE):

  #create the directory
  os.makedirs (os.path.dirname(JSON_FILE), exist_ok = True)

  # confirm and save an empty list on the file
  print(f'[i]{JSON_FILE}not found, saving empty list on file')
  with open(JSON_FILE, 'w') as file:
    json.dump([], file)

else:
  #ifnrom the user if the file already exists
  print(f'[i]{JSON_FILE} already exists.')


[i]/Data/results_SC_Sushi.jsonnot found, saving empty list on file


In [ ]:
#load previous results and set offset based on the number of results
with open (JSON_FILE, 'r') as file:
  previous_results = json.load (file)

n_results = len(previous_results)

print(f'{n_results}previous results found')

0previous results found


In [ ]:
results = yelp_api.search_query(location = LOCATION,
                                term = TERM,
                                offset = n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [ ]:
#return how many results were found
total_results = results ['total']
total_results

110

In [ ]:
business_results = results ['businesses']

#specify the filename where you want to save the data
json_file_path = JSON_FILE

In [ ]:
# save the business data
with open (json_file_path, "w") as file:
  json.dump(business_results, file, indent = 4)
  #business_data was replaced with business_results

In [ ]:
#indent = 4 means
# save te date in this format
# key: value,
# key: value,
# key: value,
# key: value,
#kasi sabi ko = 4



In [ ]:
#how many details we get?
results_per_page = len(business_results)

#business_data was replaced with business results
print(f'number of results retrieved per page', results_per_page)


number of results retrieved per page 20


In [ ]:
# use the math.ceil to round up the total number of pages
# n_pages = math.ceil(total_results/results_per_page)
# print(f'total number of pages: {n_pages}')
if results_per_page > 0:
    n_pages = math.ceil(total_results / results_per_page)
    print(f'Total number of pages: {n_pages}')
else:
    print("Error: The number of results per page must be greater than 0.")
# this fixes the div by 0 error

Total number of pages: 6


In [ ]:
# create a loop to extract the data
for i in tqdm_notebook(range(1, total_results + 1)):
  try:
    time.sleep(0.2) # short delay to respect API rate limits

    # load existing results to append new data
    with open(JSON_FILE, 'r') as file:
      previous_results = json.load(file)

    # Fetch new results
    new_results = yelp_api.search_query(location = LOCATION,
                                        term = TERM,
                                        offset = len(previous_results))

    # append and save the updated results
    updated_results = previous_results + new_results['businesses']
    with open(JSON_FILE, 'w') as file:
      json.dump(updated_results, file)

  except Exception as e:
    if 'Too Many Requests for url' in str(e):
      print('Rate limit exceeded. Stopping data collection')
      break # exit the loop if the rate limit is exceeded

    else:
      print(f'An error occured: {e}')
      continue # continue to the next iteration in case of other errors

  0%|          | 0/110 [00:00<?, ?it/s]

In [ ]:
# load the final JSON file
df = pd.read_json(JSON_FILE)

# display the first 5 rows of the data
df.head()

In [ ]:
# specify the directory
directory = '/content/drive/MyDrive/Colab Notebooks/Data'
filename = 'final_results_SC_suschi.csv.gz' # include the .csv.gz extension
path = os.path.join(directory, filename)

# ensure that the 'Data' directory exists
os.makedirs(directory, exist_ok=True)

# save the dataframe as a compressed csv file (to save space)
df.to_csv(path, compression='gzip', index=False)

In [ ]:
# save as JSON file
json_file = '/content/drive/MyDrive/Colab Notebooks/Data/final_results_SC_sushi.json'

# save the json as a dataframe
df.to_json(json_file, orient='records', lines=True)

In [ ]:
# save the file as gzip
csv_gz_file = json_file.replace('.json', '.csv.gz')

# save the dataframe as a compressed csv without the index
df.to_csv(csv_gz_file, compression='gzip', index=False)